# IMD3004 - IA Generativa

### Professor: Dr. Leonardo Enzo Brito da Silva

### Aluno: João Antonio Costa Paiva Chagas

## Tarefa:

Utilize a base de dados de dígitos MNIST.

1. Modifique a arquitetura do gerador base (reduzindo sua capacidade) mantendo o discriminador base.

por exemplo
- removendo camadas
- reduzindo o número de kernels por camada

2. Modifique a arquitetura do discriminador base (reduzindo sua capacidade) mantendo o gerador base.

por exemplo:
- removendo camadas
- reduzindo o número de kernels por camada

3. Varie as taxas de aprendizado do gerador e do discriminador de forma independente.

4. Teste um otimizador adicional (ex.: SGD ou Adam).

5. Realize a interpolação entre dois vetores latentes.


Entregáveis:
1. Notebook
2. Relatório pdf: **Reporte e comente os resultados no relatório.**

### Importações:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

import torchvision
import torchvision.transforms as transforms

from torchvision import datasets

### Ambiente

In [ ]:
if not os.path.exists('files'):
    os.makedirs('files')

### Dados:

In [ ]:
def get_mnist_transform():
    return transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

In [ ]:
def load_mnist_dataset(transform):
    training_data = datasets.MNIST(
        root="data",
        train=True,
        download=True,
        transform=transform,
    )

    return training_data

In [ ]:
def create_dataloader(training_data, batch_size):
    train_dataloader = DataLoader(
        training_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    return train_dataloader

In [ ]:
def prepare_mnist_data(batch_size):
    transform = get_mnist_transform()
    training_data = load_mnist_dataset(transform)
    return create_dataloader(training_data, batch_size)

### Plotando:

In [ ]:
# Define a função de gerar dados falsos de um vetores latentes aleatorios
def gerar_aleatorio(gerador, device, z_dim, n_images=20):
    gerador.eval()                  # Coloca o modelo em modo de avaliação (desativa dropout, batchnorm, etc.)
    z = torch.randn((n_images, z_dim), device=device)
    # Desativa o cálculo de gradientes para economizar memória e acelerar a execução
    with torch.no_grad():
      dados_falsos = modelo_gerador(z).detach().cpu().numpy()
    return dados_falsos

In [ ]:
# Função para plotar várias imagens em um grid
def plot_multiple_images(imagens):
    # Cria subplots (1 linha, -1 colunas)
    fig, axes = plt.subplots(1, len(imagens), figsize=(15, 2))

    # Itera por cada subplot e mostra a imagem
    for ax, img in zip(axes, imagens):
        ax.imshow(img, cmap='gray')    # 1 = preto, 0 = branco
        ax.axis('off')                 # Remove ticks dos eixos

    plt.subplots_adjust(wspace=0.1)    # ajusta o espacamento entre subplots

    plt.show()

In [ ]:
def interpolar_espaco_latente(gerador, z_dim, steps, device='cpu'):

    # Gera dois vetores latentes aleatórios
    z1 = torch.randn((1, z_dim), device=device)
    z2 = torch.randn((1, z_dim), device=device)

    gerador.eval()  # Coloca o gerador em modo de avaliação

    alpha_valores = torch.linspace(0, 1, steps).to(device)  # Valores de interpolação entre 0 e 1
    imagens_interpoladas = []

    for alpha in alpha_valores:
        # Interpola linearmente entre z1 e z2: z_interp = (1 - alpha) * z1 + alpha * z2 , alpha em [0, 1]
        z_interp = (1 - alpha) * z1 + alpha * z2

        with torch.no_grad():  # Evita calcular gradientes (modo de inferência)
            imagem_gerada = gerador(z_interp.to(device)).cpu()

        # Remove dimensões extras e converte para numpy
        imagens_interpoladas.append(imagem_gerada.squeeze().numpy())

    return imagens_interpoladas

### DCGAN:

#### Hiperparâmetros:

In [ ]:
def get_hyperparameters(experiment_choice):
    """
    Retorna um dicionário de hiperparâmetros baseado na escolha do experimento.

    Convenção de Nomes:
    - Otimizadores: 'Ig' (Iguais), 'Ind1' (G=SGD, D=Adam), 'Ind2' (G=Adam, D=SGD)
    - Taxas de Apr.: 'Ig' (Iguais), 'Ind1' (G<D), 'Ind2' (G>D)

    Exemplo: 'Ind1Ind2' -> G=SGD, D=Adam (Otimizadores Independentes 1) e G>D (LRs Independentes 2)

    Choices Válidas:
    - 'IgIg': G/D Adam, LRs iguais
    - 'IgInd1': G/D Adam, LR G < D
    - 'IgInd2': G/D Adam, LR G > D
    - 'Ind1Ig': G=SGD/D=Adam, LRs iguais
    - 'Ind2Ig': G=Adam/D=SGD, LRs iguais
    - 'Ind1Ind1': G=SGD/D=Adam, LR G < D
    - 'Ind2Ind1': G=Adam/D=SGD, LR G < D
    - 'Ind1Ind2': G=SGD/D=Adam, LR G > D
    - 'Ind2Ind2': G=Adam/D=SGD, LR G > D
    """

    base_params = {
        'z_size': 100,
        'num_epochs': 50,
        'batch_size': 64
    }

    if experiment_choice == 'IgIg':
        # Configuração com taxas de aprendizado iguais e otimizadores iguais
        exp_params = {
            'lr_g': 0.0002,
            'lr_d': 0.0002,
            'optimizer_g': 'Adam',
            'optimizer_d': 'Adam'
        }
    elif experiment_choice == 'IgInd1':
        # Teste com taxas de aprendizado independentes e otimizadores iguais
        print("Executando experimento com taxas de aprendizado diferentes.")
        exp_params = {
            'lr_g': 0.0001,  # Gerador mais lento
            'lr_d': 0.0004,  # Discriminador mais rápido
            'optimizer_g': 'Adam',
            'optimizer_d': 'Adam'
        }
    elif experiment_choice == 'IgInd2':
        # Teste com taxas de aprendizado independentes e otimizadores iguais
        print("Executando experimento com taxas de aprendizado diferentes.")
        exp_params = {
            'lr_g': 0.0004,  # Gerador mais rápido
            'lr_d': 0.0001,  # Discriminador mais lento
            'optimizer_g': 'Adam',
            'optimizer_d': 'Adam'
        }
    elif experiment_choice == 'Ind1Ig':
        # Configuração com taxas de aprendizado iguais e otimizadores independentes
        exp_params = {
            'lr_g': 0.0002,
            'lr_d': 0.0002,
            'optimizer_g': 'SGD',
            'optimizer_d': 'Adam'
        }
    elif experiment_choice == 'Ind2Ig':
        # Configuração com taxas de aprendizado iguais e otimizadores independentes
        exp_params = {
            'lr_g': 0.0002,
            'lr_d': 0.0002,
            'optimizer_g': 'Adam',
            'optimizer_d': 'SGD'
        }
    elif experiment_choice == 'Ind1Ind1':
        # Teste com taxas de aprendizado independentes e otimizadores independentes
        print("Executando experimento com taxas de aprendizado diferentes.")
        exp_params = {
            'lr_g': 0.0001,  # Gerador mais lento
            'lr_d': 0.0004,  # Discriminador mais rápido
            'optimizer_g': 'SGD',
            'optimizer_d': 'Adam'
        }
    elif experiment_choice == 'Ind2Ind1':
        # Teste com taxas de aprendizado independentes e otimizadores independentes
        print("Executando experimento com taxas de aprendizado diferentes.")
        exp_params = {
            'lr_g': 0.0001,  # Gerador mais lento
            'lr_d': 0.0004,  # Discriminador mais rápido
            'optimizer_g': 'Adam',
            'optimizer_d': 'SGD'
        }
    elif experiment_choice == 'Ind1Ind2':
        # Teste com taxas de aprendizado independentes e otimizadores independentes
        print("Executando experimento com taxas de aprendizado diferentes.")
        exp_params = {
            'lr_g': 0.0004,  # Gerador mais rápido
            'lr_d': 0.0001,  # Discriminador mais lento
            'optimizer_g': 'SGD',
            'optimizer_d': 'Adam'
        }
    elif experiment_choice == 'Ind2Ind2':
        # Teste com taxas de aprendizado independentes e otimizadores independentes
        print("Executando experimento com taxas de aprendizado diferentes.")
        exp_params = {
            'lr_g': 0.0004,  # Gerador mais rápido
            'lr_d': 0.0001,  # Discriminador mais lento
            'optimizer_g': 'Adam',
            'optimizer_d': 'SGD'
        }
    base_params.update(exp_params)
    return base_params

#### Gerador:

Gerador base:

In [ ]:
class Gerador(nn.Module):
    def __init__(self, z_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 7*7*256), # 128 para esse lab, 256 para o lab 6
            nn.Unflatten(dim=1, unflattened_size=(256, 7, 7)),
            nn.BatchNorm2d(256),
            nn.ConvTranspose2d(256, 128, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ReLU(True),
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128, 1, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z)

Gerador fraco:

In [ ]:
class GeradorFraco(nn.Module):
    def __init__(self, z_dim):
        super().__init__()
        self.net = nn.Sequential(
            # De 128 para 64 canais
            nn.Linear(z_dim, 7*7*64),
            nn.Unflatten(dim=1, unflattened_size=(64, 7, 7)),
            nn.BatchNorm2d(64),

            # De 128->64 para 64->32
            nn.ConvTranspose2d(64, 32, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ReLU(True),
            nn.BatchNorm2d(32),

            # De 64->1 para 32->1
            nn.ConvTranspose2d(32, 1, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z)

#### Discriminador:

Discriminador base:

In [ ]:
class Discriminador(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Dropout(p=0.4),
            nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Dropout(p=0.4),
            nn.Flatten(),
            nn.Linear(7*7*128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

Discriminador fraco:

In [ ]:
class DiscriminadorFraco(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            # De 64 para 32 canais
            nn.Conv2d(1, 32, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Dropout(p=0.4),
            # De 128 para 64
            nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Dropout(p=0.4),
            nn.Flatten(),
            # De (7*7*128) para (7*7*64)
            nn.Linear(7*7*64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

#### Treinamento:

In [ ]:
def train_dcgan(dataloader, gerador, discriminador, loss_fn, opt_gerador, opt_discriminador, device, epoch, z_dim):
    gerador.train()         # Coloca o modelo em modo de treinamento (ativa dropout, batchnorm, etc., se houver)
    discriminador.train()   # Coloca o modelo em modo de treinamento (ativa dropout, batchnorm, etc., se houver)

    size = len(dataloader.dataset)

    for batch, (real_data, _) in enumerate(dataloader):

        batch_size = real_data.shape[0]
        real_data = real_data.to(device)

        # ========================================
        # Treinamento do DISCRIMINADOR
        # ========================================

        # Gera rótulos para dados reais (1) e falsos (0)
        labels_reais = torch.ones((batch_size, 1), device=device)
        labels_falsos = torch.zeros((batch_size, 1), device=device)

        # Gera dados falsos a partir de ruído aleatório (distribuicão normal)
        z = torch.randn((batch_size, z_dim), device=device)
        dados_falsos = gerador(z)

        # Calcula a saída do discriminador para dados reais e falsos
        saida_reais = discriminador(real_data)
        saida_falsos = discriminador(dados_falsos.detach())  # detach() evita que os gradientes fluam para o gerador

        # Calcula a perda do discriminador
        perda_reais = loss_fn(saida_reais, labels_reais)
        perda_falsos = loss_fn(saida_falsos, labels_falsos)
        perda_discriminador = perda_reais + perda_falsos

        # Atualiza o discriminador
        opt_discriminador.zero_grad()
        perda_discriminador.backward()
        opt_discriminador.step()

        # ========================================
        # Treinamento do GERADOR
        # ========================================

        # Gera novos vetores latentes
        z = torch.randn((batch_size, z_dim), device=device)
        dados_falsos = gerador(z)

        # Queremos que o discriminador pense que os dados falsos são reais
        saida_falsos = discriminador(dados_falsos)
        perda_gerador = loss_fn(saida_falsos, labels_reais)  # usamos labels_reais aqui!

        # Atualiza o gerador
        opt_gerador.zero_grad()
        perda_gerador.backward()
        opt_gerador.step()

        # ========================================
        # Log de progresso
        # ========================================
        if batch % 100 == 0 or batch == len(dataloader) - 1:
            print(f"[Época {epoch:03d}] [Lote {batch:03d}/{len(dataloader)}] "
                  f"Perda D: {perda_discriminador.item():.4f} | Perda G: {perda_gerador.item():.4f}")


In [ ]:
# Define a função de gerar dados falsos de um vetor latente fixo
def gerar(gerador, device, z_dim=100):
    gerador.eval()                  # Coloca o modelo em modo de avaliação (desativa dropout, batchnorm, etc.)
    # Desativa o cálculo de gradientes para economizar memória e acelerar a execução
    with torch.no_grad():
      z = torch.zeros((1, z_dim), device=device)
      dados_falsos = gerador(z).detach().cpu().numpy().reshape(28,28)
    return dados_falsos

In [ ]:
def training_loop(dataloader, gerador, discriminador, opt_gerador, opt_discriminador, device, z_dim, num_epochs, lr_gerador, lr_discriminador):
    evolução_dado_falso = []
    loss_fn = nn.BCELoss()
    if opt_gerador == 'Adam':
        opt_gerador = torch.optim.Adam(gerador.parameters(), lr=lr_gerador, betas=(0.5, 0.999))
    elif opt_gerador == 'SGD':
        opt_gerador = torch.optim.SGD(gerador.parameters(), lr=lr_gerador, momentum=0.9)

    if opt_discriminador == 'Adam':
        opt_discriminador = torch.optim.Adam(discriminador.parameters(), lr=lr_discriminador, betas=(0.5, 0.999))
    elif opt_discriminador == 'SGD':
        opt_discriminador = torch.optim.SGD(discriminador.parameters(), lr=lr_discriminador, momentum=0.9)

    for epoca in range(1, num_epochs + 1):
        train_dcgan(
            dataloader=dataloader,
            gerador=modelo_gerador,
            discriminador=modelo_discriminador,
            loss_fn=loss_fn,
            opt_gerador=opt_gerador,
            opt_discriminador=opt_discriminador,
            device=device,
            epoch=epoca,  # Pass the current epoch number
            z_dim=z_dim
        )
        if epoca % 5 == 0 or epoca == 1 or epoca == num_epochs:
          evolução_dado_falso.append(gerar(gerador=modelo_gerador, device=device, z_dim=z_dim))

    return evolução_dado_falso

### Salvando, carregando:

In [ ]:
def save_model(model, name):
    """Salva o modelo do gerador treinado num arquivo."""
    model_path = f'files/mnist_{name}_conv_dcgan.pt'
    torch.jit.script(model).save(model_path)
    print(f"\nModelo final salvo em: {model_path}")
    return model_path

In [ ]:
def load_model(model_path, device):
    """Carrega um modelo salvo e o configura para o modo de avaliação."""
    loaded_model = torch.jit.load(model_path, map_location=device)
    loaded_model.eval()
    return loaded_model

### Executando:

#### 1:

In [ ]:
# Pega os hiperparâmetros (otimizadores iguais, taxas de aprendizado iguais)
params = get_hyperparameters('IgIg')

# Prepara os dados
train_loader = prepare_mnist_data(params['batch_size'])

# Cria os modelos de gerador e discriminador
modelo_gerador = GeradorFraco(params['z_size'])
modelo_discriminador = Discriminador()

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
modelo_gerador.to(device)
modelo_discriminador.to(device)

# Treinando
evolucao = training_loop(
    train_loader,
    modelo_gerador,
    modelo_discriminador,
    params['optimizer_g'],
    params['optimizer_d'],
    device,
    params['z_size'],
    params['num_epochs'],
    params['lr_g'],
    params['lr_d']
    )

# Desenhando
plot_multiple_images(evolucao)

# Salvando
save_model(modelo_gerador, 'weaker_generator')

#### 2:

In [ ]:
# Pega os hiperparâmetros (otimizadores iguais, taxas de aprendizado iguais)
params = get_hyperparameters('IgIg')

# Prepara os dados
train_loader = prepare_mnist_data(params['batch_size'])

# Cria os modelos de gerador e discriminador
modelo_gerador = Gerador(params['z_size'])
modelo_discriminador = DiscriminadorFraco()

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
modelo_gerador.to(device)
modelo_discriminador.to(device)

# Treinando
evolucao = training_loop(
    train_loader,
    modelo_gerador,
    modelo_discriminador,
    params['optimizer_g'],
    params['optimizer_d'],
    device,
    params['z_size'],
    params['num_epochs'],
    params['lr_g'],
    params['lr_d']
    )

# Desenhando
plot_multiple_images(evolucao)

# Salvando
save_model(modelo_gerador, 'weaker_discriminator')

#### 3:

In [ ]:
# Pega os hiperparâmetros (Gerador mais lento, discriminador mais rápido, otimizadores iguais)
params = get_hyperparameters('IgInd1')

# Prepara os dados
train_loader = prepare_mnist_data(params['batch_size'])

# Cria os modelos de gerador e discriminador
modelo_gerador = Gerador(params['z_size'])
modelo_discriminador = Discriminador()

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
modelo_gerador.to(device)
modelo_discriminador.to(device)

# Treinando
evolucao = training_loop(
    train_loader,
    modelo_gerador,
    modelo_discriminador,
    params['optimizer_g'],
    params['optimizer_d'],
    device,
    params['z_size'],
    params['num_epochs'],
    params['lr_g'],
    params['lr_d']
    )

# Desenhando
plot_multiple_images(evolucao)

# Salvando
save_model(modelo_gerador, 'slower_generator_quicker_discriminator')

In [ ]:
# Pega os hiperparâmetros (Gerador mais rápido, discriminador mais lento, otimizadores iguais)
params = get_hyperparameters('IgInd2')

# Prepara os dados
train_loader = prepare_mnist_data(params['batch_size'])

# Cria os modelos de gerador e discriminador
modelo_gerador = Gerador(params['z_size'])
modelo_discriminador = Discriminador()

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
modelo_gerador.to(device)
modelo_discriminador.to(device)

# Treinando
evolucao = training_loop(
    train_loader,
    modelo_gerador,
    modelo_discriminador,
    params['optimizer_g'],
    params['optimizer_d'],
    device,
    params['z_size'],
    params['num_epochs'],
    params['lr_g'],
    params['lr_d']
    )

# Desenhando
plot_multiple_images(evolucao)

# Salvando
save_model(modelo_gerador, 'quicker_generator_slower_discriminator')

#### 4:

In [ ]:
# Pega os hiperparâmetros (Gerador com otimizador SGD, discriminador com otimizador Adam, taxas de aprendizado iguais)
params = get_hyperparameters('Ind1Ig')

# Prepara os dados
train_loader = prepare_mnist_data(params['batch_size'])

# Cria os modelos de gerador e discriminador
modelo_gerador = Gerador(params['z_size'])
modelo_discriminador = Discriminador()

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
modelo_gerador.to(device)
modelo_discriminador.to(device)

# Treinando
evolucao = training_loop(
    train_loader,
    modelo_gerador,
    modelo_discriminador,
    params['optimizer_g'],
    params['optimizer_d'],
    device,
    params['z_size'],
    params['num_epochs'],
    params['lr_g'],
    params['lr_d']
    )

# Desenhando
plot_multiple_images(evolucao)

# Salvando
save_model(modelo_gerador, 'sgd_generator_adam_discriminator')

In [ ]:
# Pega os hiperparâmetros (Gerador com otimizador Adam, discriminador com otimizador SGD, taxas de aprendizado iguais)
params = get_hyperparameters('Ind2Ig')

# Prepara os dados
train_loader = prepare_mnist_data(params['batch_size'])

# Cria os modelos de gerador e discriminador
modelo_gerador = Gerador(params['z_size'])
modelo_discriminador = Discriminador()

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
modelo_gerador.to(device)
modelo_discriminador.to(device)

# Treinando
evolucao = training_loop(
    train_loader,
    modelo_gerador,
    modelo_discriminador,
    params['optimizer_g'],
    params['optimizer_d'],
    device,
    params['z_size'],
    params['num_epochs'],
    params['lr_g'],
    params['lr_d']
    )

# Desenhando
plot_multiple_images(evolucao)

# Salvando
save_model(modelo_gerador, 'adam_generator_sgd_discriminator')

#### 5:

In [ ]:
base_gen = load_model('files/mnist_base_conv_dcgan.pt', device)
imagens_base_generator_model = interpolar_espaco_latente(base_gen, 100, 20, device)
plot_multiple_images(imagens_base_generator_model)

In [ ]:
weaker_gen = load_model('files/mnist_weaker_generator_conv_dcgan.pt', device)
imagens_weaker_generator_model = interpolar_espaco_latente(weaker_gen, 100, 20, device)
plot_multiple_images(imagens_weaker_generator_model)

In [ ]:
weaker_disc = load_model('files/mnist_weaker_discriminator_conv_dcgan.pt', device)
imagens_weaker_discriminator_model = interpolar_espaco_latente(weaker_disc, 100, 20, device)
plot_multiple_images(imagens_weaker_discriminator_model)

In [ ]:
slow_quick = load_model('files/mnist_slower_generator_quicker_discriminator_conv_dcgan.pt', device)
imagens_slow_quick_model = interpolar_espaco_latente(slow_quick, 100, 20, device)
plot_multiple_images(imagens_slow_quick_model)

In [ ]:
quick_slow = load_model('files/mnist_quicker_generator_slower_discriminator_conv_dcgan.pt', device)
imagens_quick_slow_model = interpolar_espaco_latente(quick_slow, 100, 20, device)
plot_multiple_images(imagens_quick_slow_model)

In [ ]:
sgd_adam = load_model('files/mnist_sgd_generator_adam_discriminator_conv_dcgan.pt', device)
imagens_sgd_adam_model = interpolar_espaco_latente(sgd_adam, 100, 20, device)
plot_multiple_images(imagens_sgd_adam_model)

In [ ]:
adam_sgd = load_model('files/mnist_adam_generator_sgd_discriminator_conv_dcgan.pt', device)
imagens_adam_sgd_model = interpolar_espaco_latente(adam_sgd, 100, 20, device)
plot_multiple_images(imagens_adam_sgd_model)

#### Para o lab 6:

In [ ]:
# Pega os hiperparâmetros
params = get_hyperparameters('IgIg')

# Prepara os dados
train_loader = prepare_mnist_data(params['batch_size'])

# Cria os modelos de gerador e discriminador
modelo_gerador = Gerador(params['z_size'])
modelo_discriminador = Discriminador()

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
modelo_gerador.to(device)
modelo_discriminador.to(device)

# Treinando
evolucao = training_loop(
    train_loader,
    modelo_gerador,
    modelo_discriminador,
    params['optimizer_g'],
    params['optimizer_d'],
    device,
    params['z_size'],
    params['num_epochs'],
    params['lr_g'],
    params['lr_d']
    )

# Desenhando
plot_multiple_images(evolucao)

# Salvando
save_model(modelo_gerador, 'base')

In [ ]:
from torchsummary import summary

params = get_hyperparameters('IgIg')
train_loader = prepare_mnist_data(params['batch_size'])

modelo_gerador = Gerador(params['z_size'])
modelo_discriminador = Discriminador()

summary(modelo_gerador, input_size=(params['z_size'],))
summary(modelo_discriminador, input_size=(1, 28, 28))